# A cautionary tale of ground state energy calculation

When calculating the ground state energy of an electronic structure Hamiltonian, some care should be exercised. Converting the Hamiltonian to a sparse matrix and finding the least eigenvalue will not always be the _true_ ground state with respect to the system it represents, since the diagonalization scheme used may not inherently observe any symmetries present in the underlying physical system.

Consider the following $H_3^+$ example, consisting of 2 electrons in 6 spin-orbitals...

In [1]:
import numpy as np
import os
import json
from matplotlib import pyplot as plt

cwd = os.getcwd()
notebook_dir =  os.path.dirname(cwd)
symmer_dir = os.path.dirname(notebook_dir)
test_dir = os.path.join(symmer_dir, 'tests')
ham_data_dir = os.path.join(test_dir, 'hamiltonian_data')

if not os.path.isdir(ham_data_dir):
    raise ValueError('cannot find data dir')
    
filename = 'H3+_STO-3G_SINGLET_JW.json'

if filename not in os.listdir(ham_data_dir):
    raise ValueError('unknown file')
    
with open(os.path.join(ham_data_dir, filename), 'r') as infile:
    data_dict = json.load(infile)

In [7]:
from symmer.symplectic import PauliwordOp

fci_energy = data_dict['data']['calculated_properties']['FCI']['energy']
hf_array   = np.asarray(data_dict['data']['hf_array'])
n_particles = data_dict['data']['n_particles']
number_operator = PauliwordOp.from_dictionary(data_dict['data']['auxiliary_operators']['number_operator'])
H = PauliwordOp.from_dictionary(data_dict['hamiltonian'])

Naively computing the ground state energy by taking the smallest eigenvalue of the Hamiltonian does not match the FCI energy:

In [8]:
from symmer.utils import exact_gs_energy

gs_nrg, psi = exact_gs_energy(H.to_sparse_matrix)

print(f'Least eigenvalue = {gs_nrg} | FCI error = {gs_nrg - fci_energy}')

Least eigenvalue = -1.3609986808737295 | FCI error = -0.08658598114704441


What has gone wrong here? Taking a look at the corresponding eigenvector, we see the identified state actually contains 3 particles, whereas the underlying system only contains 2:

In [9]:
print(f'Eigenvector with eigenvalue {gs_nrg}:\n')
print(psi)

Eigenvector with eigenvalue -1.3609986808737295:

-0.000+0.000j |000111> +
-0.124+0.001j |001011> +
-0.000-0.000j |001101> +
 0.000+0.000j |001110> +
-0.000+0.000j |010011> +
 0.000+0.000j |010110> +
-0.000-0.000j |011001> +
 0.000-0.000j |011100> +
-0.150+0.002j |100011> +
-0.000-0.000j |100110> +
 0.000+0.000j |101001> +
 0.150-0.002j |101100> +
 0.000+0.000j |110000> +
 0.000+0.000j |110001> +
-0.000-0.000j |110010> +
 0.000-0.000j |110100> +
 0.969-0.010j |111000>


To counter this issue, we instead need to select the least eigenvalue that contains the _correct_ number of particles. This is implemented in `symmer.chemistry.exact_gs_energy`

In [11]:
gs_nrg, psi = exact_gs_energy(
    H.to_sparse_matrix, 
    n_particles=n_particles, 
    number_operator=number_operator
)

print(f'Least eigenvalue = {gs_nrg} | FCI error = {gs_nrg - fci_energy}\n')
print(f'Eigenvector with eigenvalue {gs_nrg}:\n')
print(psi)

Least eigenvalue = -1.274412699726865 | FCI error = -1.7985612998927536e-13

Eigenvector with eigenvalue -1.274412699726865:

 0.096-0.006j |000011> +
 0.096-0.006j |001100> +
 0.000-0.000j |011000> +
 0.000-0.000j |100100> +
-0.989+0.063j |110000> +
-0.000+0.000j |110001> +
 0.000+0.000j |111000>


Success! We have now recovered the true ground state by enforcing that only solutions with the correct number of particles are considered. Note however it is possible that no solution is identified at first - in this case, increase the `n_eigs` parameter in `symmer.chem.exact_gs_state` to increase the search space.